In [1]:
import torch
import torch.nn as nn

# random_matrix = torch.randn(7, 7)
# # QR 分解得到正交矩阵 Q
# Q, _ = torch.linalg.qr(random_matrix, mode='complete')
# print(Q)
Q=torch.eye(7)
torch.save(Q, 'identity_matrix.pt')

In [8]:
# import torch
# #print(torch.rand(7))
# # 设置缩放因子范围（可以根据需要调整）
# scale_min, scale_max = 0.5, 2.0

# # 生成两个随机缩放因子
# scale_factors = torch.rand(7) * (scale_max - scale_min) + scale_min  # 范围 [scale_min, scale_max]

# # 创建一个对角矩阵，缩放因子在对角线上
# scale_matrix = torch.diag(scale_factors)

# print("Random scale matrix:", scale_matrix)
# torch.save(scale_matrix, 'scale_matrix.pt')

Random scale matrix: tensor([[1.8575, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.2748, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0942, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9125, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.3832, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6908, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0347]])


In [2]:
import torch
import torch.nn as nn

class Affine(nn.Module):
    def __init__(self, dim):
        super(Affine, self).__init__()
        
        # 生成随机正交矩阵 A（使用 QR 分解）
        #random_matrix = torch.randn(dim, dim)
        # QR 分解得到正交矩阵 Q
        #Q, _ = torch.linalg.qr(random_matrix, mode='complete')  # 使用 torch.linalg.qr
        Q= torch.load('identity_matrix.pt')
        S=torch.load('identity_matrix.pt')
        Q=torch.matmul(Q,S)
        # 将 A 和 b 作为常量属性存储，并确保它们不会被训练
        self.register_buffer('A', Q)  # register_buffer 确保 A 不可训练
        self.register_buffer('b', torch.zeros(dim))  # 随机平移向量 b，不可训练

    def forward(self, x, *args, **kwargs):
        # 仿射变换 y = A * x + b
        x=x.view(-1, self.A.shape[1])
        y=torch.matmul(x,self.A) + self.b
        y=y.view(-1, 6, self.A.shape[0])
        return y

In [3]:
transformation= Affine(7)

torch.save(transformation.state_dict(), 'plain.pth')

In [15]:
transformation= Affine(7)
a=torch.randn(64,6,7)
b=transformation(a)
print(b.shape)

torch.Size([64, 6, 7])


In [5]:
state=torch.load('affine.pth')
transformation.load_state_dict(state)

<All keys matched successfully>